In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

import os
os.environ["WANDB_DISABLED"] = "true"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
! nvcc --version
!nvidia-smi -L
device

In [ ]:
!pip install -U transformers
#!pip install transformers==4.28.0
!pip install datasets
!pip install snscrape
!pip install --upgrade accelerate
!pip install datasets evaluate
!pip install accelerate -U

from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
import numpy as np
import random
import csv
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
import datasets
from datasets import Dataset, DatasetDict
import evaluate
from sklearn.metrics import classification_report, f1_score

import sys, os
import torch
import json
import re

In [ ]:
# preprocess functions

def remove_url(text):
    return ' '.join(re.sub("(\w+:\/\/\S+)"," ", text).split())

def remove_username(text):
    return ' '.join(re.sub("([@][A-Za-z0-9_]+)"," ", text).split())

def remove_at_mark(text):
    return re.sub(r'[@]', ' ', text)

def remove_tag_mark(text):
    return re.sub(r'[#]', ' ', text)

def replace_hashtags_with_segments (text):
   regex = "#(\w+)"
   text_split = text.split()
   sent= ''
   strip_word =' '
   for word in text_split:
     hashtag= re.findall(regex, word)
     #print(hashtag)
     #print(strip_word)

     if (len(hashtag)) == 0 :
        sent = sent + ' ' + word
     if (len(hashtag))!=0:
        strip_word = word.replace(('#'+hashtag[0]), ' ')
        #print (hashtag[0])
        #print(strip_word)
        try :
           key = '#'+ hashtag[0]
           segmented = segmented_dict [key]
           if type(segmented) == str:
                    #segmented = re.sub(r'[|]', ' ', segmented)
                    #segmented = re.sub(r'[,]', ' ', segmented)
                    sent= sent + ' ' + segmented
           if type(segmented) == list:
                    for k in segmented:
                      sent = sent +' ' + k
        except:
           sent = sent + ' ' +  hashtag[0]
   return sent + ' ' + strip_word

def control_tag_mark(text):
     s = ' '
     for word in text.split():
            word = re.sub(r'[#]', ' #', word)
            s = s + ' ' + word
     return s

def remove_punctuation_marks(text):
   punc = '''!()-[]{};:'"\,<>./?$%^&*~'''
   for i in text:
       if i in punc:
         text = text.replace(i, " ")
   return text

def remove_underline(text):
  punc = '''_'''
  for i in text:
       if i in punc:
         text = text.replace(i, "")
  return text

def remove_hashtag(text):
    return ' '.join(re.sub("([#][A-Za-z0-9_]+)"," ", text).split())

def remove_turkish_hashtag(text):
    return ' '.join(re.sub("([#][\w+_]+)"," ", text).split())

def find_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return re.findall(emoji_pattern, text) # no emoji

def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return (emoji_pattern.sub(r'', text)) # no emoji

def remove_arabic_hashtags(text):
   return ' '.join(re.sub('([#][\u0600-\u06ff_]+)', " ", text).split())

def remove_emoji_duplicate (text):
      emoji =[]
      rest = ''
      main_text = []
      for w in text :
         x = find_emoji(w)
         if len (x) != 0 :
            emoji.append (x[0])
      #print(emoji)
      rest = remove_emoji(text)
      emoji = list(set(emoji))
      for e in emoji :
        try:
          rest = rest + " " + e
        except:
          rest = rest
      main_text.append(rest)
      return main_text[0]

def lower_case(text):
    text = text.replace("I", "ı")
    text = text.replace("İ", "i")
    return text.lower()

In [ ]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(0)

In [ ]:
# Define data path
# Please replace the addresses with the address of dataset on your local drive

path_turkish_data     =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/df_turkish_cats_targets_degrees_final_with_translated_all.csv'
path_arabic_data      =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/df_arabic_cats_targets_degrees_final.csv'
path_synthetic_data   =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/synthetic_data_all_new.xlsx'

path_all_emoji =         '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/all_emoji_dict.json'
path_hashtag_segment  =  '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/data/all_hashtag_segmented_tr.json'

In [ ]:
# Laod emoji_dict and hashtag_dict

f = open(path_all_emoji)
data = json.load(f)
emoji_list = data['emojis']

f = open(path_hashtag_segment)
segmented_dict = json.load(f)

In [ ]:
# Select which preprocess steps must be apply on data

def pre_process_tweets(text_list):
   cleaned_text =[]
   for s in text_list:
       #s = lower_case(s)
       s = remove_username(s)
       s = remove_at_mark(s)
       s = remove_url(s)
       s = remove_tag_mark(s)
       #s = control_tag_mark(s)
       s = remove_punctuation_marks(s)
       s = remove_underline(s)
       #s = replace_hashtags_with_segments(s)
       cleaned_text.append(s)
   return cleaned_text

In [ ]:
#  Main target groups in the dataset

main_targets = {'Hedef grup belirgin değil veya yok.': 0,
                'Ülke/Milliyet': 1,
                'Irk/Etnik Köken': 2,
                'Din': 3,
                'Cinsiyet': 4,
                'Cinsel Yönelim': 5,
                'Belli Görüş/Statü/Uygulama; Mesleki Pozisyon Grubu': 6,
                'Hedef grup birden fazla.': 7}

In [ ]:
# Labels after merging :

# 'Ülke/Milliyet' and 'Irk/Etnik Köken'  merged
# 'Cinsiyet' and 'Cinsel Yönelim' merged

labels = ['Target group not specified or not present' ,
          'Country/Nationality/Race/Ethnicity' ,
          'Religion',
          'Gender/Sexual Orientation',
          'Specific Viewpoint/Status/Practice; Occupational Position Group']


map_7_to_5 = { 0:0, 1:1, 2:1, 3:2, 4:3, 5:3 , 6:4}
def extract_5class(inp):
    out = set()
    for i in inp:
      out.add(map_7_to_5[i])

    return out


# remove 0 class (no-target), when there is other targets

def conflicting_target_0(inp):
    out = inp.copy()
    if 0 in inp and len(inp) > 1:
        out.remove(0)
    return out

In [ ]:
#Load synthetic data

df_syn = pd.read_excel(path_synthetic_data)
df_syn = df_syn.iloc[:,[4,8,9]]


df_syn = df_syn[df_syn.TargetAll != set()].copy()
df_syn['TargetAll']= df_syn.TargetAll.apply(eval)

print(df_syn['TargetAll'].value_counts())

df_syn['TargetAll'] = df_syn.TargetAll.apply(extract_5class)
df_syn['TargetAll'] = df_syn.TargetAll.apply(conflicting_target_0)

print(df_syn['TargetAll'].value_counts())

df_syn= df_syn.sample(frac = 1)
df_syn


In [ ]:
#Load synthetic data

df_turkish = pd.read_csv(path_turkish_data)

df_turkish = df_turkish.iloc[:,[2, 36,38, 39]]

label_all = df_turkish['TargetAll'].tolist()
label_all = [str({item}) for item in label_all]

df_turkish = df_turkish[df_turkish.TargetAll != set()].copy()
df_turkish['TargetAll']= df_turkish.TargetAll.apply(eval)

df_turkish['TargetAll'] = df_turkish.TargetAll.apply(extract_5class)
df_turkish['TargetAll'] = df_turkish['TargetAll'].apply(lambda x: x.discard(4) or x)    # ignor this class: 'Specific Viewpoint/Status/Practice; Occupational Position Group'
df_turkish['TargetAll'] = df_turkish.TargetAll.apply(conflicting_target_0)
df_turkish = df_turkish[df_turkish.TargetAll != set()].copy()

print(df_turkish['TargetAll'].value_counts())


#df_all = df_all.reset_index()
df_turkish= df_turkish.sample(frac = 1)

df_turkish

In [ ]:
import pandas as pd

df_arabic = pd.read_csv(path_arabic_data)

df_arabic = df_arabic.iloc[:,[2,36,38]]


df_arabic = df_arabic[df_arabic.TargetAll != set()].copy()
df_arabic['TargetAll']= df_arabic.TargetAll.apply(eval)

df_arabic['TargetAll'] = df_arabic.TargetAll.apply(extract_5class)
df_arabic['TargetAll'] = df_arabic['TargetAll'].apply(lambda x: x.discard(4) or x)  # ignor this class: 'Specific Viewpoint/Status/Practice; Occupational Position Group'
df_arabic['TargetAll'] = df_arabic.TargetAll.apply(conflicting_target_0)
df_arabic = df_arabic[df_arabic.TargetAll != set()].copy()

print(df_arabic['TargetAll'].value_counts())
df_arabic= df_arabic.sample(frac = 1)

df_arabic



In [ ]:
# define the name and the adress of trained model on HuggingFace

hub_id = 'HrantDinkFoundation/'
model_id = 'hs-target-prediction'

In [ ]:
# select which data used in training

# 'only-tr' : use only Turkish data
# 'only-ar' : use only arabic data
# 'tr-syn'  : use Turkish and synthetic data
# 'ar-syn'  : use arabic and synthetic data
# 'ar-translated-syn'  : use arabic, synthetic and the translated column of Turkish data

mode_train = ['only-tr', 'only-ar', 'tr-syn', 'ar-syn', 'ar-translated-syn']
mode = mode_train[2]

In [ ]:
if mode =='only-tr':
    model_lang = 'tr'
    model_name = 'dbmdz/bert-base-turkish-uncased'
    df_train = df_turkish[(df_turkish.Div=='train')]
    df_test  = df_turkish[(df_turkish.Div=='test')]


if mode =='only-ar' :
     model_lang = 'ar'
     model_name = 'aubmindlab/bert-base-arabert'
     df_train = df_arabic[(df_arabic.Div=='train')]
     df_test  = df_arabic[(df_arabic.Div=='test')]


if mode == 'tr-syn':
    model_lang = 'tr'
    model_name = 'dbmdz/bert-base-turkish-uncased'
    hub_id = hub_id + 'turkish-' + model_id
    df_train = df_turkish[(df_turkish.Div=='train')]
    df_test  = df_turkish[(df_turkish.Div=='test')]

    df_train = pd.concat([df_train, df_syn], ignore_index=True)


if mode == 'ar-syn':
    model_lang = 'ar'
    model_name = 'aubmindlab/bert-base-arabert'
    df_train = df_arabic[(df_arabic.Div=='train')]
    df_test  = df_arabic[(df_arabic.Div=='test')]

    df_syn = df_syn.drop (columns=['Text'])
    df_syn.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)
    df_train = pd.concat([df_train, df_syn], ignore_index=True)


if mode == 'ar-translated-syn' :
     model_lang = 'ar'
     model_name = 'aubmindlab/bert-base-arabert'
     hub_id = hub_id + 'arabic-' + model_id
     df_turkish = df_turkish.drop (columns=['Text'])
     df_turkish.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)

     df_syn = df_syn.drop (columns=['Text'])
     df_syn.rename(columns={'translated_text_arabic': 'Text'}, inplace=True)

     df_train = df_arabic[(df_arabic.Div=='train')]
     df_test  = df_arabic[(df_arabic.Div=='test')]

     df_train = pd.concat([df_train, df_turkish, df_syn], ignore_index=True)



In [ ]:
df_train

In [ ]:
df_test


In [ ]:
num_class = 4
column_select = 'TargetAll'
k_fold = 10


In [ ]:

df_train = df_train[df_train.TargetAll != set()].copy()

text = df_train['Text'].tolist()
label = df_train[column_select].tolist()
label = [str(x) for x in label]
text = pre_process_tweets(text)
train_data = pd.DataFrame(list(zip(text, label)), columns =['text', 'label'])

print(train_data['label'].value_counts())
train_data


In [ ]:
import ast

df_test = df_test[df_test.TargetAll != set()].copy()
print(df_test['TargetAll'].value_counts())

text = df_test['Text'].tolist()
label = df_test[column_select].tolist()
label = [list(s) for s in label]
label = list(map(lambda l: list(np.where(np.isin(np.arange(num_class), l), float(1), float(0))), label))

text = pre_process_tweets(text)
test_data = pd.DataFrame(list(zip(text, label)), columns =['text', 'label'])
#test_data= test_data.astype({'label':'int'})

print(test_data['label'].value_counts())
test_data

In [ ]:
# Load BERT model

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import ElectraForSequenceClassification,  ElectraTokenizerFast
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaTokenizerFast, RobertaForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, GPT2Config

labels = ['Target group not specified or not present' ,
          'Country/Nationality/Race/Ethnicity' ,
          'Religion',
          'Gender/Sexual Orientation']

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

pretrained_model_name = model_name
print(model_name)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, do_lower_case=True, force_download=True)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels = num_class, problem_type="multi_label_classification",
                                                           force_download=True, id2label=id2label, label2id=label2id).to(device)

model.resize_token_embeddings(len(tokenizer))
for item in emoji_list :
        tokenizer.add_tokens(item)
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
def preprocess_function(examples):
    preprocessed = examples["text"]
    return tokenizer(preprocessed, truncation=True)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    try:
      roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    except ValueError:
      pass
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
k_fold = 8
skf = StratifiedKFold(n_splits= k_fold, random_state = 31, shuffle=True)


In [ ]:
class CustomTrainer(Trainer):
    def save_model(self, output_dir=None, _internal_call=False):
        if output_dir is None:
            output_dir = self.args.output_dir

        self.model = self.model.to('cuda')

        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()

        super().save_model(output_dir, _internal_call)

In [ ]:
# Let's start training model

for i, (train_index, test_index) in enumerate(skf.split(train_data.text , train_data.label)):

    print("number of samples in train folds: ", len(train_index))
    print("number of samples in test fold  : ", len(test_index))

    test = train_data.iloc[test_index].set_index("text")
    multi_label_test = test['label'].tolist()
    multi_label_test = [list(ast.literal_eval(item)) for item in multi_label_test]
    label_test = list(map(lambda l: list(np.where(np.isin(np.arange(num_class), l), float(1), float(0))), multi_label_test))
    test = test.drop (columns=['label'])
    test['label'] = label_test
    print(label_test)
    test = Dataset.from_pandas(test)


    train = train_data.iloc[train_index].set_index("text")
    multi_label_train = train['label'].tolist()
    multi_label_train = [list(ast.literal_eval(item)) for item in multi_label_train]
    label_train = list(map(lambda l: list(np.where(np.isin(np.arange(num_class), l), float(1), float(0))), multi_label_train))
    trian = train.drop (columns=['label'])
    train['label'] = label_train
    train = Dataset.from_pandas(train)


    dataset_dict = DatasetDict({"test": test, "train": train})
    preprocessed_dataset_dict = dataset_dict.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
        output_dir= "My Hate Speech Detection",
        overwrite_output_dir=True,
        #report_to="none",
        learning_rate=5e-6,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=20,
        num_train_epochs=10,
        weight_decay=0.01,
        eval_strategy="steps",
        metric_for_best_model = 'f1',
        logging_steps = 100,
        load_best_model_at_end=True,
        greater_is_better=True,
        #hub_strategy="end",          # These parameter used when you want saved mmodels directly to HuggingFace,
        #push_to_hub=True,            # If you want to save model on huggingFace you must select "True"
        #hub_model_id= hub_id,        # Write your model address on HuggingFace
        #hub_private_repo= False,     # Determine whether the model on Hugging Face should be private or public.
        #hub_token= "...........",    # To save model into HuggingFace you need "hub_token" which can take from your own HuggingFace account
    )

    trainer = CustomTrainer(
        model = model,
        args = training_args,
        train_dataset=preprocessed_dataset_dict["train"],
        eval_dataset=preprocessed_dataset_dict["test"],
        tokenizer = tokenizer,
        data_collator = data_collator,
        compute_metrics = compute_metrics,

    )
    trainer.train()
    break

#trainer.save_model()      # If you want save your trained model on HuggingFace


In [ ]:
#Let's start testing model

test_ds = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)
test_ds

In [ ]:
#Let's start testing model

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

model.eval()

y_test = test_data['label'].tolist()
y_test = np.array(y_test)
y_pred =[]

for t in test_data['text'].tolist() :
   encoding = tokenizer(t, return_tensors="pt")
   encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

   outputs = trainer.model(**encoding)
   logits = outputs.logits
   logits.shape
   sigmoid = torch.nn.Sigmoid()
   probs = sigmoid(logits.squeeze().cpu())
   predictions = np.zeros(probs.shape)
   predictions[np.where(probs >= 0.3)] = 1
   y_pred.append(predictions)


In [ ]:
label_mapping = {0:'Target group not specified or not present' ,
                 1: 'Country/Nationality/Race/Ethnicity' ,
                 2: 'Religion',
                 3: 'Gender/Sexual Orientation'}

In [ ]:
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix

print ('-------------------------------------------------------')
print(classification_report(y_test, y_pred, digits=num_class, zero_division=0))


In [ ]:
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import confusion_matrix

print ('-------------------------------------------------------')
print(classification_report(y_test, y_pred, target_names=labels, zero_division=0))


In [ ]:
# Saving trained model on local drive
# Please replace the pathes with your local address path


if model_lang == 'tr':
          path_save_model = '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/trained_models/hs-tr-target/'

if model_lang == 'ar':
          path_save_model = '/content/gdrive/My Drive/Colab Notebooks/my_hate_speech/all_data_work/final_all_data/HS_GitHub_data_codes/trained_models/hs-ar-strength/'

trainer.save_model(path_save_model)

In [ ]:
# Load trained model from local drive
# Please add the address where the models are saved

from transformers import pipeline

path_trained_model = path_save_model
model_pipline = pipeline(task = "text-classification", model = path_trained_model, top_k=None, return_all_scores=True, device="cuda")

In [ ]:
def predict_target_tweet(review):
        prediction = []
        result = []
        predictions = model_pipline (review)
        prediction_result = predictions[0]
        result = []
        # Filter predictions based on the sigmoid threshold
        filtered_predictions = [pred for pred in prediction_result if pred['score'] > 0.2]

        # Print all labels that meet the threshold
        if filtered_predictions:
            for item in filtered_predictions:
                   result.append ((item['label'], 'score:', item['score']))
            #else:
                   #result.append ("No predictions meet the threshold.")

        return result


🔴 ▶ **Disclaimer:**

This study (project) includes examples of offensive language, hate speech, and profanity due to the nature of the research. These examples do not represent the authors’ views (or developers' views). The goal of this work is to contribute to the detection and prevention of harmful content and violence directed toward refugees, LGBTI+ individuals, religious groups, nationalities, and other vulnerable communities.

In [ ]:
AR_TWEETS_EXAMPLES = [
    "كل اللاجئين لصوص",
    "ارحلوا من بلدنا",
    "مكان المرأة هو المطبخ",
    "اليونان يجب أن تعرف حدودها، وإلا قد نأتي فجأة في ليلة",
    "لا ينبغي لغير المسلمين أن يصبحوا نوابًا.",
    "يجب ألا يُسمح للاجئين بالتصويت.",
    "يجب ألا يستفيد اللاجئون من التأمين الصحي.",
    "التعليم باللغة الكردية غير مقبول.",
    "للاجئين أيضًا الحق في الحياة.",
    "يتم اتخاذ تدابير شاملة ضمن الإطار القانوني لمنع أنشطة التنظيمات الإرهابية.",
    "تم الإعلان عن تواريخ الفعاليات الثقافية التي ستُقام هذا العام في إسرائيل.",
    "تفتتح رائدات الأعمال صالونات تجميل تستهدف العملاء من المسلمين وغير المسلمين.",
    "تُجرى الاستعدادات في الأسواق في سوريا قبل الأعياد الإسلامية والمسيحية.",
    "هل كون بعض الكلاب مسعورة أو عدوانية يبرر قتل جميع الكلاب؟ اسحبوا القانون.",
     "يجب على النساء دعم بعضهن البعض",
    "اشتريت تفاحًا من السوق.",
    "طعام السوريين لذيذ جدًا.",
    "الطعام السوري شهي للغاية.",
    "أنا أحب السوريين كثيرًا، إنهم أشخاص لطيفون جدًا.",
    "السنغافوريون يعملون بجد.",
    "أنا أحب الأمريكيين كثيرًا",
    "أنا لا أحب الملحدين على الإطلاق",
    "المسيحيين الأرثوذكس",
    "سنغافورة بلد جميل جدًا"   ,
    "ذهبت إلى المستشفى أمس، ثم تناولت وجبة طعام لذيذة، ثم عدت إلى المنزل، ولعبت مع ابنتي، وشاهدت مسلسلًا أمريكيًا",
    "أنا أحب الحيوانات كثيرًا"
]

TR_TWEETS_EXAMPLES = [
    "mültecilerin hepsi hırsızdır",
    "defolun bizim ülkemizden",
    "kadının yeri mutfaktır",
    "Yunanistan haddini bilecek yoksa bir gece ansizin gelebiliriz",
    "Gayrimüslimler milletvekili olmasın.",
    "Mülteciler oy kullanmasın.",
    "Mültecilere oy hakki verilmemeli",
    "Mülteciler sgk'dan yararlanmasın.",
    "Kürtçe eğitim kabul edilemez.",
    "Mültecilerin de yaşamaya hakkı var.",
    "Terör örgütlerinin faaliyetlerini önlemek için hukuki çerçevede kapsamlı önlemler alınmaktadır.",
    "İsrail'de bu yıl düzenlenecek kültürel etkinliklerin tarihleri açıklandı",
    "Kadın girişimciler, hem Müslüman hem de diğer topluluklardan müşterilere hitap eden güzellik salonları açıyor.",
    "Suriye’deki pazarlarda Müslüman ve Hristiyan bayramları öncesinde hazırlıklar yapılıyor.",
    "Bazı köpeklerin kuduz ya da saldırgan olması, tüm köpeklerin öldürülmesi için haklı bir sebep midir? Yasayı geri çek.",
    "Kadınlar birbirlerine destek olmalı.",
    "Pazardan elma aldim.",
    "Suriyelilerin yemekleri çok güzel.",
    "Suriye yemekleri çok lezzetli.",
    "Süriyelileri çok seviyorum, çok sempatik insanlar.",
    "Singapurlular çok çalışkan",
    "Amerikalıları çok Seviyorum",
    "Ateistleri hiç sevmiyorum",
    "Ortodoksalar hrıstiyanlar",
    "Singapur çok güzel bir ülke dir",
    "Dün hastaneye gittim sonra da bir güzel yemek yedim, eve döndüm, kızımla oyun oynadım, Amerikan dizi izledim",
    "hayvanlari çok seviyorum"
]

if model_lang == 'ar':
      TWEETS_EXAMPLES = AR_TWEETS_EXAMPLES

if model_lang == 'tr':
      TWEETS_EXAMPLES = TR_TWEETS_EXAMPLES


for tweet in TWEETS_EXAMPLES:
    target = predict_target_tweet(remove_punctuation_marks(tweet.lower()))
    print(f"Tweet:    {tweet}")
    print(f"Target: {target}")
    print("-" * 60)
